In [1]:
import pandas as pd
import prepare
import model
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)

In [2]:
df = pd.read_json('data1.json')
df = prepare.prep_repos(df)

In [3]:
target = 'language_reduced'

**Split the Data** into three separate samples to avoid data leakage while fitting and evaluating machine learning models.

In [4]:
# split the data using sklearn's train_test_split inside the split_data function
train, validate, test = prepare.split_data(df, target)

train	 n = 51
validate n = 23
test	 n = 19


# Modeling

The following code initializes the infrastructure for storing information about our models' performance

In [5]:
# initialize the model_number variable at 0
model_number = 0 
# initialize an empty dataframe
model_results = pd.DataFrame()

### Baseline

Here, we establish baseline predictions based on the most frequently ocurring target class and evaluate the baseline accuracy.

In [6]:
# create baseline predictions using the mode and get accuracy score with sklearn.metrics inside run_baseline()
model_number, model_results = model.run_baseline(train[target], model_number, model_results)

### Machine Learning Classsifiers

Here, we create, fit and evaluate the following types of machine learning classification models, using our train and validate samples. 

- Decision Tree
- Random Forest
- Multinomial Naive Bayes
- Complement Naive Bayes

Each model is run using two types of feature preprocessing:

- Term Frequency - Inverse Document Frequency (TF/IDF) Vectorization
- Count Vectorization (CV) AKA "Bag of Words"

For each type of model and type of preprocessing, the other associated hyperparameters were also varied, creating a total of 68 unique models with varying performance.

In [7]:
# user defined functions which create sklearn models with varying features and hyperparameters and store
# information about the models and their performance
model_number, model_results = model.run_decision_tree(train, validate, target, model_number, model_results)
model_number, model_results = model.run_random_forest(train, validate, target, model_number, model_results)
model_number, model_results = model.run_naive_bayes(train, validate, target, model_number, model_results)

### Results

Below, you can see the resulting accuracy scores on both the train and validate set for each model created.

In [8]:
model.display(model_results)

model_number,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,baseline
sample_type,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
train,0.862745,0.803922,0.921569,0.862745,0.960784,0.941176,1.000000,0.980392,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.647059,0.647059,0.627451,0.627451,0.627451,0.627451,0.627451,0.627451,0.666667,0.647059,0.627451,0.627451,0.627451,0.627451,0.627451,0.627451,0.666667,0.647059,0.627451,0.627451,0.627451,0.627451,0.627451,0.627451,0.666667,0.647059,0.627451,0.627451,0.627451,0.627451,0.627451,0.627451,0.960784,0.980392,1.000000,0.960784,0.627451,0.941176,0.980392,0.941176,0.627451,0.921569,0.921569,0.941176,0.627451,0.862745,0.921569,0.941176,0.627451,0.843137,0.921569,0.941176,0.627451
validate,0.434783,0.652174,0.434783,0.608696,0.434783,0.608696,0.434783,0.608696,0.434783,0.608696,0.434783,0.608696,0.434783,0.608696,0.434783,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,0.608696,NaN


### Best Model

Here, we choose the model which performed with the highest level of **accuracy** on the validate sample. We choose accuracy as our evaluation metric because in this multi-class classifier, we have no reason to favor any one of the target classes over any other.

In [9]:
# obtain the model numbers with the highest accuracy on validate
best = model.get_best(model_results)
# display info about those models
model_results[model_results.model_number.isin(best)]

,model_number,model_type,sample_type,accuracy,feature_type,max_depth,min_samples_leaf,alpha
3,2,decision_tree,train,0.803922,CV/BOW,3.0,NaN,NaN
4,2,decision_tree,validate,0.652174,CV/BOW,3.0,NaN,NaN


Model # 2 achieves the highest validate score of 65%, though this is actually 1 point below our baseline accuracy.

### Final Test

Here, we recreate model #2 and evaluate it on our test sample, which approximates how it might perform when used on additional READMEs which were not included in our samples.

In [10]:
model.test_model_2(train, test, target)

0.631578947368421

We see another small reduction in performance to 63%, making it about 3% below the baseline of 66% - and not particularly useful.